In [ ]:
# Imports 

import json
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import sys
from sqlalchemy import inspect
import pandas as pd
import json
import sys
import os
ROOT_PATH = "/Users/xaviborrat/Documents/GitHub/TFM_Clinical_Deterioration"

sys.path.append(ROOT_PATH)
#importació llibreria de classes de connexio
import classes
from classes.xavi_con_class import db_connect as xcc

In [ ]:
# Connector instantiation.


config_path = os.path.join(ROOT_PATH, 'classes/config_tfm.json')
with open(config_path, 'r') as configDnx:
    configDnx = json.load(configDnx)

datanex= xcc(
    ssh = configDnx["ssh"],
    ssh_user = configDnx["ssh_user"],
    ssh_host = configDnx["ssh_host"],
    ssh_pkey = configDnx["ssh_pkey"],
    db_host = configDnx["db_host"],
    db_port = configDnx["db_port"],
    db_user = configDnx["db_user"],
    db_pass = configDnx["db_pass"],
    flavour = configDnx["flavour"],
    db = configDnx["db"]
)


In [ ]:

# VITALS DICTIONARY 

# Vitals dictionary extraction from DataNex.
Feature_values = datanex.query('datascope4',"""Select * from dic_rc_text 
                                where rc_sap_ref in (
                                'ESTADO_CONCIENC',
                                'CAR_RITM',
                                'EPOC',
                                'O2_DIS'
                                )""") 
# Load esp_eng dictionary
esp_eng = pd.read_csv(ROOT_PATH+'/playground/values descr spanish english.csv')

# Merge the two DataFrames
values_dict = Feature_values.merge(esp_eng, left_on='descr', right_on='Original', how='left')
values_dict.drop(columns=['Original'], inplace=True)
# Write the DataFrame to deterioation database.
datanex.write_table_2('borrat_project', df=values_dict,table_name= 'vitals_values_dic')


Database [datascope4] session created...
<> Query Sucessful <>
Database [borrat_project] session created...
<> Table [vitals_values_dic] created <>


In [ ]:

# LAB DICTIONARY

# Load local to LOINC dictionary.
lab_dict = pd.read_csv(ROOT_PATH+'/playground/lab_dictionary.csv', on_bad_lines='skip',sep=';')
loinc=lab_dict[['lab a SAP','Loinc Methode']].drop_duplicates()
loinc.rename(columns={'lab a SAP': 'lab_sap_ref', 'Loinc Methode': 'loinc_code'}, inplace=True)

# Extract lab dictionary from DataNex.
dic_lab = datanex.query('datascope4',"""Select * from dic_lab""")
dic_lab.drop(columns=['lab_ref'], inplace=True)

# Merge loinc to DataNex Dictionary.
dic_lab_loinc = dic_lab.merge(loinc, on='lab_sap_ref', how='left')
dic_lab_loinc.drop_duplicates(inplace=True)
# Write the DataFrame to DataNex database.
datanex.write_table_2('borrat_project', df=dic_lab_loinc,table_name= 'lab_dic')

Database [datascope4] session created...
<> Query Sucessful <>
